In [1]:
import wrds
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
conn = wrds.Connection()


WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [ ]:
option_prices = conn.get_table(library='optionm', table='opprcd2022')


In [ ]:

arr = option_prices["symbol"].unique()
set([s[:4] for s in arr if pd.notna(s)])

{'FTS ',
 'GENC',
 'GURE',
 'MPAA',
 'RUI ',
 'STN ',
 'THTX',
 'TRQ ',
 'TRQ1',
 'TRX ',
 'WWR '}

In [ ]:
option_prices[option_prices["symbol"].str.contains("AAPL", na=False)]

,secid,date,symbol,symbol_flag,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,...,theta,optionid,cfadj,am_settlement,contract_size,ss_flag,forward_price,expiry_indicator,root,suffix


In [ ]:
option_prices[option_prices["symbol"] == "CAE 230120C12500"]

,secid,date,symbol,symbol_flag,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,...,theta,optionid,cfadj,am_settlement,contract_size,ss_flag,forward_price,expiry_indicator,root,suffix
0,5139.0,2023-01-03,CAE 230120C12500,1,2023-01-20,<NA>,C,12500.0,5.9,8.7,...,-12.42606,150842156.0,1.0,0.0,100.0,0,<NA>,<NA>,<NA>,<NA>
66,5139.0,2023-01-04,CAE 230120C12500,1,2023-01-20,<NA>,C,12500.0,6.1,9.3,...,-10.51813,150842156.0,1.0,0.0,100.0,0,<NA>,<NA>,<NA>,<NA>
132,5139.0,2023-01-05,CAE 230120C12500,1,2023-01-20,<NA>,C,12500.0,5.5,9.1,...,-10.74709,150842156.0,1.0,0.0,100.0,0,<NA>,<NA>,<NA>,<NA>
198,5139.0,2023-01-06,CAE 230120C12500,1,2023-01-20,<NA>,C,12500.0,5.8,9.4,...,-8.577157,150842156.0,1.0,0.0,100.0,0,<NA>,<NA>,<NA>,<NA>
264,5139.0,2023-01-09,CAE 230120C12500,1,2023-01-20,<NA>,C,12500.0,7.0,9.8,...,-32.08586,150842156.0,1.0,0.0,100.0,0,<NA>,<NA>,<NA>,<NA>
330,5139.0,2023-01-10,CAE 230120C12500,1,2023-01-20,<NA>,C,12500.0,6.5,9.8,...,-13.5205,150842156.0,1.0,0.0,100.0,0,<NA>,<NA>,<NA>,<NA>
396,5139.0,2023-01-11,CAE 230120C12500,1,2023-01-20,<NA>,C,12500.0,6.7,10.1,...,-22.04406,150842156.0,1.0,0.0,100.0,0,<NA>,<NA>,<NA>,<NA>
462,5139.0,2023-01-12,CAE 230120C12500,1,2023-01-20,<NA>,C,12500.0,7.3,11.1,...,-18.23684,150842156.0,1.0,0.0,100.0,0,<NA>,<NA>,<NA>,<NA>
528,5139.0,2023-01-13,CAE 230120C12500,1,2023-01-20,<NA>,C,12500.0,7.5,10.9,...,-46.40506,150842156.0,1.0,0.0,100.0,0,<NA>,<NA>,<NA>,<NA>
594,5139.0,2023-01-17,CAE 230120C12500,1,2023-01-20,<NA>,C,12500.0,7.4,9.1,...,<NA>,150842156.0,1.0,0.0,100.0,0,<NA>,<NA>,<NA>,<NA>
